# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants

import os
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Initializing a Workspace

In [2]:
ws = Workspace.from_config()

TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

## Creating an AML Experiment

In [3]:
# choose a name for experiment
experiment_name = 'automl-heart-failure'
experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()

## Creating a Compute Cluster

In [4]:
amlcompute_cluster_name = "hf-compute"

try:
    compute_target = ComputeTarget(workspace = ws, name = amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_DS3_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data Wrangling

## Data Gathering

### Overview
In this project, the Heart Failure Prediction dataset from Kaggle is used. The description of the dataset is provided below.

**The dataset has the following features:**  
* age: Age (numeric)
* anaemia: Decrease of red blood cells or hemoglobin (boolean)
* creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
* diabetes: If the patient has diabetes (boolean)
* ejection_fraction: Percentage of blood leaving the heart at each contraction (percentage)
* high_blood_pressure: If the patient has hypertension (boolean)
* platelets: Platelets in the blood (kiloplatelets/mL)
* serum_creatinine: Level of serum creatinine in the blood (mg/dL)
* serum_sodium: Level of serum sodium in the blood (mEq/L)
* sex: Woman or man (binary)

**The task:**  
Developing a ML model to predict death events using 12 clinical features.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
### Gathering the Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
data_path = "https://raw.githubusercontent.com/ugururesin/Azure-ML-Capstone-Project/main/dataset/heart_failure_clinical_records_dataset.csv"
data = TabularDatasetFactory.from_delimited_files(path = data_path)

## Data Assesing

In [6]:
df = data.to_pandas_dataframe()
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [33]:
#Creating training and testing datasets
x = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

train_df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
64,45.0,0,582,0,80,0,263358.03,1.18,137,0,0,63,0
160,66.0,1,72,0,40,1,242000.00,1.20,134,1,0,121,0
178,63.0,1,122,1,60,0,267000.00,1.20,145,1,0,147,0
176,69.0,0,1419,0,40,0,105000.00,1.00,135,1,1,147,0
292,52.0,0,190,1,38,0,382000.00,1.00,140,1,1,258,0


In [21]:
#Since pandas dataframe is not supported, the dataset needs to be converted to tabular dataset
if not os.path.isdir('dataset'):
    os.mkdir('dataset')

pd.DataFrame(train_df).to_csv("dataset/train_data.csv", index=False)
pd.DataFrame(test_df).to_csv("dataset/test_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir = './dataset', target_path = 'heart-failure-dataset/train_data.csv')

train_data = Dataset.Tabular.from_delimited_files(path = ds.path('heart-failure-dataset/train_data.csv'))

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [28]:
train_data

{
  "source": [
    "('workspaceblobstore', 'heart-failure-dataset/train_data.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "ee9ef731-d878-408d-b64e-a29485319efc",
    "name": null,
    "version": null,
    "workspace": "Workspace.create(name='quick-starts-ws-190058', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-190058')"
  }
}

### Checking the target feature

In [8]:
df['DEATH_EVENT'].value_counts()

0    203
1     96
Name: DEATH_EVENT, dtype: int64

In [9]:
get_primary_metrics("classification")

['precision_score_weighted',
 'norm_macro_recall',
 'average_precision_score_weighted',
 'accuracy',
 'AUC_weighted']

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

* **experiment_timeout_minutes**: 15 minutes is set since the dataset is small.

* **iterations**: 50 is set since the dataset is small.  

* **max_concurrent_iterations**: Represents the maximum number of iterations that would be executed in parallel.

* **n_cross_validations**: To avoid overfitting and ensure randomness.

* **primary_metric**: Since the target response is imbalanced, accuracy is not a good evaluation metric. So, auc is selected.  

* **task**: Classification is selected since the response variable is binary (0 or 1).  

* **enable_early_stopping**: Activated to avoid overfitting.

In [30]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,
    "iterations": 50,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'AUC_weighted',
    "featurization" : 'auto',
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task = "classification", 
    training_data = train_data,
    label_column_name="DEATH_EVENT",
    enable_early_stopping= True,
    debug_log = "automl_errors.log",
    compute_target = compute_target,
    **automl_settings
)
                             

In [31]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on hf-compute with default configuration
Running on remote compute: hf-compute
The validation results are as follows:
Y is empty


ValidationException: ValidationException:
	Message: Failed to execute the requested operation: data/settings validation. Error details: Validation error(s): [{
    "additional_properties": {
        "debugInfo": null
    },
    "code": "UserError",
    "severity": 2,
    "message": "Y is empty",
    "message_format": "{dataName} is empty",
    "message_parameters": {
        "dataName": "Y"
    },
    "reference_code": null,
    "details_uri": null,
    "target": "training_data",
    "details": [
        {
            "additional_properties": {
                "debugInfo": null
            },
            "code": null,
            "severity": null,
            "message": "null",
            "message_format": null,
            "message_parameters": {},
            "reference_code": null,
            "details_uri": null,
            "target": null,
            "details": [],
            "inner_error": null,
            "additional_info": null
        }
    ],
    "inner_error": {
        

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()

In [12]:
best_metrics = best_run.get_metrics()

print("Best run id : {}".format(best_run.id), 
      "AUC : {}".format(best_metrics['accuracy']), 
      "Best metrics : {}".format(best_metrics), 
      "Best model : {}".format(fitted_model), sep = '\n')

NameError: name 'best_run' is not defined

In [ ]:
#the fitted model
_, fitted_model = remote_run.get_output()
print(fitted_model)

In [ ]:
#TODO: Save the best model
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')
joblib.dump(fitted_model, filename= "outputs/automl_model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# downloading score.py and env file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH,'automl_env.yml')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# registering the model
model_name = best_run.properties['model_name']
description = 'AutoML model trained on heart failure data so as to predict whether death event occurs based on the clinical values'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id) # This will be written to the script file later in the notebook.

In [ ]:
# creating a randomly selected test data
test_data = df.sample(10)
test_label = test_data.pop('DEATH_EVENT')

# converting the records to a json data file
data_json = test_data.to_dict(orient='records')
data = json.dumps({'data': data_json})

In [ ]:
# getting inference
output = webservice.run(data)
print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
webservice.get_logs()

In [ ]:
webservice.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
